### 📘 Lesson 4: Building a PyPSA Model

<div style="display: flex; align-items: center; justify-content: space-between;">
  <div>
    <h3>Course presenters</h3>
    <ul>
      <li><strong>Priyesh Gosai</strong> - Energy Systems Modeler and Training Coordinator</li>
      <li><strong>Dr. Fabian Hofmann</strong> - Senior Optimization and Energy System Modelling Expert</li>
    </ul>
  </div>
  <div>
    <a href="https://openenergytransition.org/index.html">
      <img src="https://openenergytransition.org/assets/img/oet-logo-red-n-subtitle.png" height="60" alt="OET">
    </a>
  </div>
</div>


##### 🎯 Learning Objectives  



* Introduce participants to the PyPSA toolbox.  
* Provide details of relevant components.  
* Build and solve a simple PyPSA model.  
* Review the data structures for static and time-series data.  
* Analyze the results.  

The content also includes references to other toolboxes such as `numpy`, `pandas`, `matplotlib`, and `plotly`, but only covers functions relevant to a PyPSA workflow.  

📌 Participants unfamiliar with these toolboxes are encouraged to explore online videos or courses for deeper learning. 🎥📚  

---

### 📄 **Case Study**

We want to model the interaction between South Africa and Lesotho. 

South Africa's generation capacity has been aggregated into six generation technologies (coal,solar,wind, OCGT, hydro and battery) and a single load,and lesotho only has a load. 

### 📥 **Importing Essential Libraries**  


In [ ]:
# install_colab_dependencies() # For Google Colab users
# from google.colab import drive
# drive.mount('/content/drive')

import pypsa
import pandas as pd
import numpy as np
from training_scripts import *


### **🔧⚡ Create a PyPSA Network Object**

In [ ]:
network = pypsa.Network()

Look at what is in the network.

In [ ]:
network.all_components

Look at the component attributes.

In [ ]:
network.component_attrs['Bus']

In [ ]:
network.component_attrs['Generator']

Each component is also given a label `list_name` which is the label used to access the dataset. 

In [ ]:
network.components

In [ ]:
for key in network.component_attrs:
    print(f'{key.ljust(20)} {network.components[key]["list_name"]}')


To access the dataset: 

In [ ]:
network.buses

In [ ]:
network.generators

In [ ]:
network.links

In [ ]:
network.loads

#### 📂 Data Structure Guidelines  



✅ **Static Data:**  
- The Excel file should be configured using the `list_name` as the `sheet name`.  
- Use ``variables`` in the header.  

✅ **Time-Series Data:**  
- Follow the naming convention:  `[list]-[variable name]`
- Example: `loads-p_set` for the set loads.

---


In [ ]:
network.generators_t.p_max_pu

---
### 🐍 Add data to the network

#### ⏳ **Set up the simulation timeseries**


A `snapshot` represents a specific point in time for which the network is simulated.  
- Snapshots can be single timestamps (e.g., `2025-01-01 00:00`) or time series covering hours, days, weeks, or years.  
- They allow modeling of dynamic power system behavior over different time periods.  

📆 **Selecting the Simulation Period**

To define the simulation period, we assign snapshots to the network using a `start_time` and `end time`.

In [42]:
# Define your start and end date
start_date = '2025-01-01 00:00'
end_date = '2025-01-01 23:00'  # inclusive

# Generate hourly snapshots and set the snapshots
snapshots = pd.date_range(start=start_date, end=end_date, freq='4h')
network.set_snapshots(snapshots)


In [43]:
network.snapshots

DatetimeIndex(['2025-01-01 00:00:00', '2025-01-01 04:00:00',
               '2025-01-01 08:00:00', '2025-01-01 12:00:00',
               '2025-01-01 16:00:00', '2025-01-01 20:00:00'],
              dtype='datetime64[ns]', name='snapshot', freq='4h')

#### **🛠️🏗️ Adding Components to a PyPSA Network**

In PyPSA, the `network.add()` method is used to add components (such as buses, generators, lines, etc.) to the network. The general syntax follows:

```python
network.add(component, **attributes)
```

**Parameters**
- `component` *(str)*: The type of component to add (e.g., `"Bus"`, `"Generator"`, `"Load"`, `"Line"`, `"Link"`, etc.).
- `attributes` *(dict)*: A set of keyword arguments specifying the properties of the component.



**Adding a Carrier**
To add carriers to the network. 

```python
network.add("Carrier", name=['carrier x','carrier y','carrier z'])  
```

_Exercise: Add the following carriers_
* _AC_
* _coal_
* _gas_
* _solar_
* _wind_

In [ ]:
network.carriers

**Adding a Bus**
To add a new bus to the network:

```python
network.add("Bus", name="bus1")  # Adding a bus named 'bus1'
```

_Exercise: Add the following bus_
* _ZA_

In [ ]:
# Add bus



**⚡ Generators**

Generators attach to a single bus, converting energy from their `carrier` to the bus `carrier`.  

* Their power output is constrained by `p_nom * p_max_pu` and `p_nom * p_min_pu`.  

* Static limits define dispatchable generators, while time-varying limits model renewables.  

* Time series `p_max_pu` and `p_min_pu` determine availability per snapshot.  

* For unit commitment constraints, refer to the PyPSA documentation. 

Some key variables relevant to this model are given below. 


| Attribute              | Type            | Unit           | Default | Description | Constraint |
|------------------------|----------------|---------------|---------|-------------|------------------|
| `name`              | string         | n/a           | n/a     | Unique name |  |
| `bus`               | string         | n/a           | n/a     | Name of bus to which generator is attached |  |
| `p_nom`            | float          | MW            | 0       | Nominal power for limits in optimization. |  |
| `p_nom_extendable` | boolean        |           | False   | Switch to allow capacity p_nom to be extended in optimization. | |
| `p_min_pu`        | static/series  | per unit      | n/a     | Minimum output per unit of p_nom. | $p_t \geq p_{nom}\times p_{min,pu,t}$ |
| `p_max_pu`        | static/series  | per unit      | 1       | Maximum output per unit of p_nom. | $p_t \leq p_{nom}\times p_{max,pu,t}$ |
| `p_set`           | static/series  | MW            | n/a     | Active power set point (for PF). | $p_t = p_{set}$  |
| `e_sum_min`       | float          | MWh           | -inf    | Minimum total energy produced during optimization horizon. | $\sum p_t \cdot \delta t \leq e_{\max}$
| `e_sum_max`       | float          | MWh           | inf     | Maximum total energy produced during optimization horizon. | $\sum p_t \cdot \delta t \leq e_{\max}$
| `marginal_cost`   | static/series  | currency/MWh  | n/a     | Marginal cost of production of 1 MWh. | |



**Adding Generators**

To add a generator connected to a bus:

```python
network.add("Generator", name="gen1", bus="bus1", p_nom=100, marginal_cost=50)
```
Here:
- `bus="bus1"` specifies the bus the generator is connected to.
- `p_nom=100` sets the nominal power capacity in MW.
- `marginal_cost=50` sets the generation cost per MWh.

_Exercise: Add the following generator_
* _name = "CFPP"_
* _bus ="ZA"_
* _p_nom =27000_ # MW
* _marginal_cost =500_ #R/MWh
* _carrier = "coal"_


In [ ]:
# Add Generator

_Exercise: Add the following generators using lists as inputs._
* _name = ["OCGT", "Solar PV", "Onshore Wind"]_
* _bus = ["ZA", "ZA", "ZA"]_
* _p_nom = [8000, 5000, 4000]_ # MW
* _marginal_cost = [1200, 0, 0]_ #R/MWh

**_Slack Generator_**

A slack generator in PyPSA acts as a last-resort mechanism to ensure supply and demand balance when available generation cannot meet demand. Instead of representing actual generation, it simulates unserved demand, effectively quantifying how much load is shed to maintain system feasibility. 

By assigning a very high marginal cost, the model is strongly discouraged from using it unless absolutely necessary, making it a useful proxy for system reliability and flexibility shortfalls.

A slack generator is useful in modelling because it ensures the optimization always finds a feasible solution, even under extreme conditions where supply cannot meet demand. This allows the model to continue running rather than failing due to infeasibility, providing valuable insights into when, where, and how often shortfalls occur. By tracking the amount and cost of unserved energy, planners and system operators can identify weak points in generation capacity, flexibility, or transmission infrastructure, helping to guide investment decisions and resilience planning in future system development.

We will name the slack generator as `Loadshedding Generator`.

_Exercise: Add a slack generator._

_Exercise: Add the following generator_
* _name = "Loadshedding Generator"_
* _bus ="ZA"_
* _p_nom = 1_ # MW
* _marginal_cost =500_ #R/MWh
* _carrier = "loadshedding"_
* _p_nom_extendable_ = True_

🌞 **Applying `p_max_pu` Constraint on VRE Generators**  

* Variable Renewable Energy (VRE) generators, such as solar and wind, have time-dependent availability limits.  
* The `p_max_pu` constraint, imported as a time-series dataset, determines the maximum power output at each snapshot on a per unit basis.  


Check the number of snapshots using the `len` function.
```
len(network.snapshots)
```

In [44]:
len(network.snapshots)

6

Create a set of timeseries data into a dictionary and add it into a dataframe. 

```
solar_p_max_pu = [0.0, 0.0, 0.2, 0.6, 0.3, 0] 
wind_p_max_pu = [0.6, 0.4, 0.2, 0.3, 0.1, 0.15]

p_max_pu_dict = {"Solar PV": solar_p_max_pu, "Onshore Wind": wind_p_max_pu}

# Combine into DataFrame
p_max_pu_df = pd.DataFrame(p_max_pu_dict, index=snapshots)

network.generators_t.p_max_pu = p_max_pu_df
```

 🔌 **Loads**

* A load connects to a single bus and consumes power.  
* It can represent electricity demand or other types of loads like hydrogen or heat.  
* If active power is consumed, the load draws from the bus.  
* If reactive power is consumed, the load behaves like an inductor.  
* Loads are essential for demand modeling in power system simulations. ⚡🏠  


To add a load connected to a bus:

```python
network.add("Load", name="Load 1", bus="bus1", p_set=100)
```
Here:
- `bus="bus1"` specifies the bus the generator is connected to.
- `p_set=100` sets the nominal power capacity in MW sets a constant load for all snapshots


**Check if the network is working**

`network.optimize(solver_name = highs)`


**Adding a Battery as a `storage_unit`**
```python
network.add("StoageUnit", name="StorageUnit1", bus0="bus1",  p_nom=100 , ...)
```
Here:
- `bus0="bus1"` and `bus1="bus2"` define the buses the line connects.
- `p_nom=100` sets the nominal power in MW.

**🔋Batteries**

There are two components for energy storage in PyPSA: Storage Units and Stores.  

* ⚡ Storage Unit  
   * Attaches to a single bus and is used for inter-temporal power shifting with a time-varying state of charge.  
   * The energy capacity is defined as `max_hours * nominal power (MW)`, and it includes charging/discharging efficiencies.  

* 🏭 Store  
   * Connects to a single bus and acts as a fundamental energy storage component without energy conversion.  
   * Controls and optimizes energy capacity size, but power output must be controlled using Link components.  

🔄 Key Differences  
| Feature           | Storage Unit | Store |
|------------------|-------------|-------|
| Power Control | Directly defined | Requires Links |
| Energy Capacity | Fixed as `max_hours * MW` | Optimized independently |
| Marginal Cost | Applies only to discharging | Applies to both charging & discharging |
| Energy Carrier Conversion| Possible | Not possible (inherits from bus) |

Stores are more flexible but require Links for power control, while Storage Units offer a simpler implementation for direct energy storage modeling. ⚙️🔄  

⚡ **Storage Unit**


| Attribute              | Type            | Unit           | Default | Description |
|------------------------|----------------|---------------|---------|-------------|
| `name`              | string         | n/a           | n/a     | Unique name | 
| `bus`               | string         | n/a           | n/a     | Name of bus to which generator is attached |
| `p_nom`            | float          | MW            | 0       | Nominal power for limits in optimization. | 
| `marginal_cost`   | static/series  | currency/MWh  | n/a     | Marginal cost of production of 1 MWh. | 
| `max_hours` | float | h | 1 | Maximum state of charge capacity in terms of hours at full output capacity `p_nom` |  
|`state_of_charge_initial`| float | MWh | 0  | State of charge before the snapshots in the OPF. | 
|`efficiency_store`  | static/series | per unit | 1 | Efficiency of storage on the way into the storage. | 
|`efficiency_dispatch`| static/series | per unit | 1 | Efficiency of storage on the way out of the storage. | 
| `standing_loss`| static/series | per unit | 0 | Losses per hour to state of charge. | 
| `inflow`| static/series | MW | 0 | Inflow to the state of charge, e.g. due to river inflow in hydro reservoir. |


```python
network.add("StoageUnit", name="StorageUnit1", bus="bus1",  p_nom=100 , ...)
```

_Exercise_
* _name = "BESS"_
*  _bus = "ZA"_
* _p_nom = 1000_
* _marginal_cost = 100_
* _state_of_charge_initial = 3000_
* _max_hours = 5_



🔗 **Links**

* Links enable controllable, directed power flow between two buses (`bus0 → bus1`).  
* They can have efficiency losses and marginal costs, restricting default flow to one direction.  
* For bidirectional, lossless operation, set `efficiency = 1`, `marginal_cost = 0`, and `p_min_pu = -1`.  
* Links model HVDC interconnections, converters, heat pumps, electrolysers, and other controllable power flows.  
* ⚠️ In the actual model, lines will be used instead of links for passive AC/DC transmission.  

**Adding a Transmission Line as a `link`**
```python
network.add("Link", name="link1", bus0="bus1", bus1="bus2", p_nom=100)
```
Here:
- `bus0="bus1"` and `bus1="bus2"` define the buses the line connects.
- `p_nom=100` sets the nominal power in MW.

_Exercise: Set up a bus in Lesotho (LS) with a constant load of 80 MW and connect Lesotho to South Africa (ZA) a 100MW link wit_



---
### 🌐 Working with the `network` object

The network contains functions, such as: 

- 📥 Adding data: `network.add()` or `network.import_from_csv()` - As described before.
- :tick `network.consistency_check()` to check network consistency
- 🔍 Optimization: `network.optimize()` – Runs the optimization process.  
   * Supports multiple solvers including GLPK, Gurobi, CPLEX, and HiGHS. 
- 📊 Statistics: `network.statistics()` – Generates system-wide statistics.  
- 🗺️ Visualization: `network.plot()` – Plots the network layout.  


**Review Input Data**

In [ ]:
network.generators_t.p_max_pu.head()

In [ ]:
network.loads_t.p_set.head()

In [ ]:
network.loads

In [ ]:
network.links.head()

In [ ]:
network.consistency_check()

**Solve Model**

In [ ]:
network.optimize(solver_name='highs')

**Results**

In [ ]:
network.generators_t.p.head() # or .plot()

In [ ]:
network.storage_units_t.p.head() # or .plot()

### 
---